In [ ]:
import geopandas as gpd
import folium
from branca.colormap import linear

# ✅ 정확한 SHP 경로
shp_path = "/home/sgcho/code/gongmo/src/notebooks/nunininu/BND_ADM_DONG_PG/BND_ADM_DONG_PG.shp"

# 이름 매핑
name_mapping = {
    "광장동": "광장동", "구의1동": "구의제1동", "구의2동": "구의제2동", "구의3동": "구의제3동",
    "군자동": "군자동", "능동": "능동", "자양1동": "자양제1동", "자양2동": "자양제2동",
    "자양3동": "자양제3동", "자양4동": "자양제4동", "중곡1동": "중곡제1동", "중곡2동": "중곡제2동",
    "중곡3동": "중곡제3동", "중곡4동": "중곡제4동", "화양동": "화양동"
}

# 데이터 로드 및 변환
gdf = gpd.read_file(shp_path, encoding="cp949")
gdf = gdf[gdf["ADM_NM"].isin(name_mapping.keys())]
gdf = gdf.to_crs(epsg=4326)
gdf["행정동"] = gdf["ADM_NM"].map(name_mapping)

# 비율 데이터
gdf["2~30대여성비율"] = gdf["행정동"].map({
    "화양동": 0.35, "군자동": 0.20, "중곡제1동": 0.21,
    "중곡제2동": 0.18, "중곡제3동": 0.16, "중곡제4동": 0.14,
    "능동": 0.23, "광장동": 0.11, "자양제1동": 0.18,
    "자양제2동": 0.15, "자양제3동": 0.14, "자양제4동": 0.17,
    "구의제1동": 0.22, "구의제2동": 0.14, "구의제3동": 0.16
})

gdf["1인가구비율"] = gdf["행정동"].map({
    "화양동": 0.83, "군자동": 0.63, "중곡제1동": 0.58,
    "중곡제2동": 0.50, "중곡제3동": 0.54, "중곡제4동": 0.42,
    "능동": 0.61, "광장동": 0.20, "자양제1동": 0.56,
    "자양제2동": 0.42, "자양제3동": 0.30, "자양제4동": 0.56,
    "구의제1동": 0.60, "구의제2동": 0.39, "구의제3동": 0.37
})

gdf["노인비율"] = gdf["행정동"].map({
    "화양동": 0.11, "군자동": 0.19, "중곡제1동": 0.19,
    "중곡제2동": 0.19, "중곡제3동": 0.22, "중곡제4동": 0.23,
    "능동": 0.17, "광장동": 0.13, "자양제1동": 0.19,
    "자양제2동": 0.20, "자양제3동": 0.19, "자양제4동": 0.22,
    "구의제1동": 0.16, "구의제2동": 0.20, "구의제3동": 0.18
})



# 공통 시각화 함수
def make_map(column, caption, filename):
    m = folium.Map(location=[37.552, 127.085], zoom_start=14)
    cmap = linear.Reds_09.scale(gdf[column].min(), gdf[column].max())
    cmap.caption = caption

    folium.GeoJson(
        gdf,
        style_function=lambda feat: {
            "fillColor": cmap(feat["properties"][column]),
            "color": "black",
            "weight": 1.5,
            "fillOpacity": 0.5,
        },
        tooltip=folium.GeoJsonTooltip(fields=["행정동", column], aliases=["행정동:", caption])
    ).add_to(m)
    cmap.add_to(m)
    m.save(filename)
    print(f"✅ 저장 완료: {filename}")

# 1. 여성 비율
make_map("2~30대여성비율", "2~30대 여성 인구 비율", "gwangjin_2030female_ratio_map.html")

# 2. 1인가구 비율
make_map("1인가구비율", "1인가구 비율", "gwangjin_single_ratio_map.html")

# 3. 노인 비율
make_map("노인비율", "노인 인구 비율", "gwangjin_elderly_ratio_map.html")


✅ 저장 완료: gwangjin_female_ratio_map.html
✅ 저장 완료: gwangjin_single_ratio_map.html
✅ 저장 완료: gwangjin_elderly_ratio_map.html
